This notebook is mostly to take a look at the data, and to figure what kinds of things are described in it.

In [ ]:
from pyspark.sql import SparkSession


def get_spark():
    return SparkSession.builder.config("spark.local.dir", "/tmp").getOrCreate()


spark = get_spark()
spark

: 

In [ ]:
print("hello")

: 